This is a classification example to show how to use Oboe for training and testing, in the context of AutoML, i.e., do pipeline selection on the training set and then evaluate the performance of the selected model on the test set.

In [1]:
# necessary modules
import sys
import pandas as pd
import os
import time
import numpy as np
import multiprocessing

#import AutoLearner module by either specifying its relative path or doing pip installation
automl_path = '../automl'
sys.path.append(automl_path)
from auto_learner import AutoLearner
import util

#import scikit-learn modules
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# disable warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#load and split dataset into training and test folds
data = load_iris()
x = np.array(data['data'])
y = np.array(data['target'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
categorical = [False for _ in range(x.shape[1])] # a Boolean list of feature types: categorical or not

# Example 1: a no-brainer use

In [3]:
# initialize the autolearner class
m = AutoLearner(p_type='classification', runtime_limit=100, method='TensorOboe', verbose=True)

shape of the error tensor: (551, 4, 2, 2, 8, 183)
Loading latent factors from storage ...
Loading saved runtime predictors ...


In [4]:
# fit autolearner on training set and record runtime
start = time.time()
m.fit(x_train, y_train, categorical) # TensorOboe accepts the list of feature types
elapsed_time = time.time() - start

Fitted an ensemble with size 1
having a capped running time of 32 seconds
Fitted an ensemble with size 1
Fitted an ensemble with size 1
Fitted an ensemble with size 1
Fitted an ensemble with size 1
Fitted an ensemble with size 1
Fitting a candidate learners not fitted before ..
Fitting a candidate learners not fitted before ..
Fitting a candidate learners not fitted before ..
Fitting a candidate learners not fitted before ..
Fitting a candidate learners not fitted before ..
Fitting a candidate learners not fitted before ..
Fitting a candidate learners not fitted before ..
Fitting a candidate learners not fitted before ..
Fitting a candidate learners not fitted before ..
cv errors: [0.03532407 0.04989418 0.05244009 0.05244009 0.06401416 0.03532407
 0.05244009 0.06401416 0.04917639 0.09327692 0.04845199 0.04917639]
Fitted an ensemble with size 5
having a capped running time of 47 seconds
cv errors: [0.03532407 0.04989418 0.05244009 0.05244009 0.06401416 0.03532407
 0.05244009 0.06401416 

In [5]:
# use the fitted autolearner for prediction on test set
y_predicted = m.predict(x_test)
print("prediction error: {}".format(util.error(y_test, y_predicted, 'classification')))    
print("elapsed time: {}".format(elapsed_time))

prediction error: 0.08421052631578946
elapsed time: 21.445039749145508


In [6]:
# get names of the selected machine learning models
m.get_models()

{'ensemble method': 'select at most 5 pipelines with smallest cv error',
 'base learners': [{'imputer': {'algorithm': 'SimpleImputer',
    'hyperparameters': {'strategy': 'most_frequent'}},
   'encoder': {'algorithm': None},
   'standardizer': {'algorithm': None},
   'dim_reducer': {'algorithm': 'SelectKBest', 'hyperparameters': {'k': 1}},
   'estimator': {'algorithm': 'lSVM', 'hyperparameters': {'C': 2}}},
  {'imputer': {'algorithm': 'SimpleImputer',
    'hyperparameters': {'strategy': 'most_frequent'}},
   'encoder': {'algorithm': None},
   'standardizer': {'algorithm': None},
   'dim_reducer': {'algorithm': 'SelectKBest', 'hyperparameters': {'k': 1}},
   'estimator': {'algorithm': 'lSVM', 'hyperparameters': {'C': 2}}},
  {'imputer': {'algorithm': 'SimpleImputer',
    'hyperparameters': {'strategy': 'most_frequent'}},
   'encoder': {'algorithm': None},
   'standardizer': {'algorithm': None},
   'dim_reducer': {'algorithm': 'SelectKBest', 'hyperparameters': {'k': 1}},
   'estimator': 

# Example 2: build an ensemble of models with given configurations

In [7]:
#experimental settings
VERBOSE = False #whether to print out information indicating current fitting progress
N_CORES = 1 #number of cores
RUNTIME_BUDGET = 15

In [8]:
# #optional: limit the types of algorithms (not yet supported)
# s = ['AB', 'ExtraTrees', 'GNB', 'KNN', 'RF', 'DT']

In [9]:
#autolearner arguments
autolearner_kwargs = {
    'p_type': 'classification',
    'method': 'TensorOboe',
    'runtime_limit': RUNTIME_BUDGET,
    'verbose': VERBOSE,
    'selection_method': 'min_variance',
    'stacking_alg': 'greedy',
    'n_cores': N_CORES,
    'build_ensemble': True,
}

In [10]:
#intialize the autolearner class
m = AutoLearner(**autolearner_kwargs)

In [11]:
# fit autolearner on training set and record runtime
start = time.time()
m.fit(x_train, y_train, categorical)
elapsed_time = time.time() - start

In [12]:
# use the fitted autolearner for prediction on test set
y_predicted = m.predict(x_test)
print("prediction error: {}".format(util.error(y_test, y_predicted, 'classification')))
print("elapsed time: {}".format(elapsed_time))

prediction error: 0.042105263157894736
elapsed time: 3.519043207168579


In [13]:
# get names of the selected machine learning models
m.get_models()

{'ensemble method': 'select at most 5 pipelines with smallest cv error',
 'base learners': [{'imputer': {'algorithm': 'SimpleImputer',
    'hyperparameters': {'strategy': 'constant'}},
   'encoder': {'algorithm': None},
   'standardizer': {'algorithm': None},
   'dim_reducer': {'algorithm': 'SelectKBest', 'hyperparameters': {'k': 3}},
   'estimator': {'algorithm': 'GBT',
    'hyperparameters': {'learning_rate': 0.025,
     'max_depth': 3,
     'max_features': 'log2'}}}]}